In [4]:
import geopandas as gp
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

import optuna
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np
import sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score

import os

from hyperparam_opt import *

In [8]:
pwd


'/mnt/c/Users/2toth/Desktop/546/project/2022-Project-Repo/serf'

In [18]:
ls

 Locations.csv*                          Untitled.ipynb*
 Main_SERF_nb.ipynb*                     __init__.py*
'Multivariate Correlation.ipynb'*        __pycache__/
 NASA/                                   arima_dataframe.py*
 Plotly.ipynb*                           arima_yearly_averages.py*
 Prophet-tech-review/                   'hyperparam all files.ipynb'*
 SARIMAX-EXOG.ipynb*                     hyperparam_opt.py*
 SERF-model-walkthru.ipynb*              hyperparam_opt_guide.ipynb*
 SERF.ipynb*                             json_to_csv.py*
 SERF_environment.yml*                   json_to_csv_test.py*
 TS-train-test-split.py*                 tests/
'Testing my patience good lord.ipynb'*   unittesting-testing-grounds.ipynb*
 Train-test-post-optimization.ipynb*


In [99]:
arr = os.listdir('../../hyperparameters/')

arr2 = np.argsort(arr)
arr
#arr = arr.sort()
#arr.sort(key=lambda f: int(filter(str.isdigit, f)))
#arr = sorted(arr)
print(os.path.splitext(arr[1])[0][31:])

def filenumber(x):
    x = os.path.splitext(x)[0]
    return(25 - int(x[31:]))

arr = sorted(arr, key = filenumber) 
#print(int(os.path.splitext(arr[4])[0].split('SARIMAX_hyperparameter_results_')[1]))
#arr = sorted(arr,key=lambda x: int(os.path.splitext(arr[x])[0].split('SARIMAX_hyperparameter_results_')[1]))
arr

1


['SARIMAX_hyperparameter_results_25.csv',
 'SARIMAX_hyperparameter_results_24.csv',
 'SARIMAX_hyperparameter_results_23.csv',
 'SARIMAX_hyperparameter_results_22.csv',
 'SARIMAX_hyperparameter_results_21.csv',
 'SARIMAX_hyperparameter_results_20.csv',
 'SARIMAX_hyperparameter_results_19.csv',
 'SARIMAX_hyperparameter_results_18.csv',
 'SARIMAX_hyperparameter_results_17.csv',
 'SARIMAX_hyperparameter_results_16.csv',
 'SARIMAX_hyperparameter_results_15.csv',
 'SARIMAX_hyperparameter_results_14.csv',
 'SARIMAX_hyperparameter_results_13.csv',
 'SARIMAX_hyperparameter_results_12.csv',
 'SARIMAX_hyperparameter_results_11.csv',
 'SARIMAX_hyperparameter_results_10.csv',
 'SARIMAX_hyperparameter_results_9.csv',
 'SARIMAX_hyperparameter_results_8.csv',
 'SARIMAX_hyperparameter_results_7.csv',
 'SARIMAX_hyperparameter_results_6.csv',
 'SARIMAX_hyperparameter_results_5.csv',
 'SARIMAX_hyperparameter_results_4.csv',
 'SARIMAX_hyperparameter_results_3.csv',
 'SARIMAX_hyperparameter_results_2.csv',


In [100]:
for k in range(len(arr)):
    arr[k] = '../../hyperparameters/'+ arr[k]

In [103]:
arr

['../../hyperparameters/SARIMAX_hyperparameter_results_25.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_24.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_23.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_22.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_21.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_20.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_19.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_18.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_17.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_16.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_15.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_14.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_13.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_12.csv',
 '../../hyperparameters/SARIMAX_hyperparameter_results_11.csv',
 '../../hyperparameters/SARIMAX_hyperpar

In [108]:
df = pd.read_csv(arr[0])
df = df.sort_values('value',ascending=False)
df.head(1)

,Unnamed: 0,number,value,datetime_start,datetime_complete,duration,params_D,params_P,params_Q,params_d,params_p,params_q,state
31,31,31,0.681055,2022-03-10 01:31:52.914199,2022-03-10 01:31:56.629975,0 days 00:00:03.715776,1,1,0,1,3,3,COMPLETE


,lat,lon,Place,p
0,49.0362,-122.3247,"Abbotsford, Canada",
1,49.0362,-118.4245,"Grand Forks, Canada",
2,48.9289,-117.8306,"Northport, WA",
3,48.7835,-122.3357,"Dewey, WA",
4,48.7826,-121.7994,"Mt. Baker, WA",
5,48.5767,-122.9509,"Shaw Island, WA",
6,48.5085,-117.3252,"Rudy, WA",
7,48.4093,-123.5716,"Metchosin, Canada",
8,48.0139,-123.4453,"Port Angeles, WA",
9,47.9735,-122.2038,"Everett, WA",


In [127]:
hyperparam=[]

for k in range(len(arr)):
    df = pd.read_csv(arr[k])
    df = df.sort_values('value',ascending=False)
    hyperparam.append(df[['params_D','params_P','params_Q','params_d','params_p','params_q']].head(1).values[0])
hyperparam.append([0,0,0,0,0,0])
hyperparam = np.array(hyperparam)
hyperparam = np.array(hyperparam)
hyperparam

array([[1, 1, 0, 1, 3, 3],
       [1, 0, 1, 0, 7, 8],
       [0, 2, 0, 1, 6, 7],
       [1, 1, 0, 0, 8, 6],
       [0, 0, 2, 1, 8, 8],
       [1, 1, 0, 0, 7, 7],
       [1, 1, 2, 1, 8, 5],
       [1, 1, 0, 1, 7, 4],
       [1, 0, 0, 1, 7, 8],
       [1, 0, 0, 0, 4, 5],
       [1, 1, 3, 0, 2, 5],
       [0, 2, 3, 0, 7, 7],
       [1, 0, 0, 0, 5, 6],
       [1, 1, 1, 1, 6, 1],
       [1, 1, 0, 0, 7, 2],
       [1, 2, 3, 0, 5, 7],
       [0, 3, 3, 0, 6, 6],
       [1, 2, 3, 0, 7, 5],
       [1, 0, 0, 0, 1, 7],
       [0, 1, 1, 0, 6, 7],
       [1, 0, 0, 0, 2, 5],
       [1, 0, 3, 1, 8, 1],
       [1, 0, 1, 0, 5, 8],
       [1, 0, 0, 0, 4, 5],
       [1, 0, 0, 0, 0, 6],
       [0, 2, 2, 0, 6, 7],
       [0, 0, 0, 0, 0, 0]])

In [129]:
dfloc = pd.read_csv('locations.csv')
dfloc['p'] = hyperparam[:,4]
dfloc['d'] = hyperparam[:,3]
dfloc['q'] = hyperparam[:,5]
dfloc['P'] = hyperparam[:,1]
dfloc['D'] = hyperparam[:,0]
dfloc['Q'] = hyperparam[:,2]
dfloc

,lat,lon,Place,p,d,q,P,D,Q
0,49.0362,-122.3247,"Abbotsford, Canada",3,1,3,1,1,0
1,49.0362,-118.4245,"Grand Forks, Canada",7,0,8,0,1,1
2,48.9289,-117.8306,"Northport, WA",6,1,7,2,0,0
3,48.7835,-122.3357,"Dewey, WA",8,0,6,1,1,0
4,48.7826,-121.7994,"Mt. Baker, WA",8,1,8,0,0,2
5,48.5767,-122.9509,"Shaw Island, WA",7,0,7,1,1,0
6,48.5085,-117.3252,"Rudy, WA",8,1,5,1,1,2
7,48.4093,-123.5716,"Metchosin, Canada",7,1,4,1,1,0
8,48.0139,-123.4453,"Port Angeles, WA",7,1,8,0,1,0
9,47.9735,-122.2038,"Everett, WA",4,0,5,0,1,0


In [130]:
dfloc.to_csv('locations.csv')